In [4]:
# %% Imports
import os
from dask import dataframe as ddf
from dask.distributed import LocalCluster, progress
import multiprocessing as mp


In [5]:
# %% Generate partitioned dataset

cluster = LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()), # 90% of CPU cores
    processes=True, # Use processes instead of threads
    threads_per_worker=1, # One thread per worker
    memory_limit='2GB', # Memory limit per worker
    #n_workers=4, threads_per_worker=2
)
client = cluster.get_client()  # set up local cluster
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 7
Total threads: 7,Total memory: 13.04 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64786,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:64807,Total threads: 1
Dashboard: http://127.0.0.1:64817/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:64789,


2025-11-26 12:42:13,076 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1101ea074d12efe3fcea34a98d262a1c initialized by task ('shuffle-transfer-1101ea074d12efe3fcea34a98d262a1c', 37) executed on worker tcp://127.0.0.1:64831
2025-11-26 12:43:03,064 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1101ea074d12efe3fcea34a98d262a1c deactivated due to stimulus 'task-finished-1764178983.0548675'
2025-11-26 13:03:31,600 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1101ea074d12efe3fcea34a98d262a1c initialized by task ('shuffle-transfer-1101ea074d12efe3fcea34a98d262a1c', 37) executed on worker tcp://127.0.0.1:64818
2025-11-26 13:04:30,396 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1101ea074d12efe3fcea34a98d262a1c deactivated due to stimulus 'task-finished-1764180270.3910666'


In [19]:
# %% Read Parquet
import os
os.chdir("C:\\Temp")

df = ddf.read_parquet(
    "bigData",
    dtype_backend="pyarrow"
)

In [21]:
# %% Transform Data

df = df.groupby("station").agg(["min", "max", "mean"])
df = df.sort_values("station").compute()
progress(df)

VBox()

2025-11-26 13:04:30,743 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x0000021A4A721E80>>, <Task finished name='Task-1097785' coro=<MultiProgressBar.listen() done, defined at C:\Users\VHAATGGoldsD15\AppData\Local\miniforge3\envs\dask\Lib\site-packages\distributed\diagnostics\progressbar.py:281> exception=AttributeError("'MultiProgressWidget' object has no attribute 'elapsed_time'")>)
Traceback (most recent call last):
  File "C:\Users\VHAATGGoldsD15\AppData\Local\miniforge3\envs\dask\Lib\site-packages\tornado\ioloop.py", line 758, in _run_callback
    ret = callback()
  File "C:\Users\VHAATGGoldsD15\AppData\Local\miniforge3\envs\dask\Lib\site-packages\tornado\ioloop.py", line 782, in _discard_future_result
    future.result()
    ~~~~~~~~~~~~~^^
  File "C:\Users\VHAATGGoldsD15\AppData\Local\miniforge3\envs\dask\Lib\site-packages\distributed\diagnostics\progressb

In [27]:
df.to_csv("df_out.csv")#, engine="pyarrow")

In [22]:
df.head()

measure                 
                min   max       mean
station                             
Abha          -34.6  66.4  17.998542
Abidjan       -20.0  75.2  25.985385
Abéché        -23.8  77.5  29.413701
Accra         -20.5  80.1  26.394294
Addis Ababa   -34.7  65.9  15.998364

In [28]:
# %% close local cluster
client.close()
cluster.close()